# Imports and Boilerplate

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
from jump_detection.processor import TimeSeriesProcessor

# Initialization


In [4]:
# Define a window size for the moving Fstat
window_size = 5
gap_size = 0
processor = TimeSeriesProcessor(window_size, gap_size)
segments = processor.process_data("example_data.csv")


In [5]:
# Now segments contains a list of Segment objects.
# We can loop through the segments and print them out
for i, segment in enumerate(segments[0:2]):
    print(f"Segment {i}:")
    print("Original:")
    print(segment.original)
    print("Fstats:")
    print(segment.Fstats)
    print("\n")

Segment 0:
Original:
          0         1         2         3         4
0  0.736350  0.142999  0.425887  0.604167  0.550780
1  0.675804  0.352918  0.900349  0.988412  0.079779
2  0.702146  0.589975  0.066487  0.614260  0.724433
3  0.655386  0.437358  0.464970  0.634625  0.552566
4  0.996404  0.063771  0.472241  0.405533  0.900574
5  0.617503  0.530586  0.588521  0.483992  0.312746
Covariance:
          0         1         2         3         4
0  0.736350  0.142999  0.425887  0.604167  0.550780
1  0.675804  0.352918  0.900349  0.988412  0.079779
2  0.702146  0.589975  0.066487  0.614260  0.724433
3  0.655386  0.437358  0.464970  0.634625  0.552566
4  0.996404  0.063771  0.472241  0.405533  0.900574
5  0.617503  0.530586  0.588521  0.483992  0.312746


Segment 1:
Original:
           0         1         2         3         4
1   0.675804  0.352918  0.900349  0.988412  0.079779
2   0.702146  0.589975  0.066487  0.614260  0.724433
3   0.655386  0.437358  0.464970  0.634625  0.552566
4   